In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML, Markdown
from matplotlib import rc
import ipywidgets as widgets
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual

def printmd(string):
    display(Markdown(string))
    
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%matplotlib inline

### Run below for only interactive graphs

In [15]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
sf_2009 = pd.read_csv('data/2009 SINGLE FAMILY.txt')
sf_2013 = pd.read_csv('data/2013 SINGLE FAMILY.txt')
sf_2017 = pd.read_csv('data/2017 SINGLE FAMILY.txt')

In [4]:
sf1 = pd.merge(sf_2009, sf_2013, on= 'AddressFullAddress', how = 'outer',suffixes = ('_f','_s'))
sf1.columns

sf1.rename(columns = {'FinishedArea_f' : '2009_FinishedArea', 'FinishedArea_s' : '2013_FinishedArea'}, inplace = True)

sf1['APN']= sf1['APN_f'].where(sf1['APN_s'].isnull(), sf1['APN_s'])
sf1['DistrictCode']= sf1['DistrictCode_f'].where(sf1['DistrictCode_s'].isnull(), sf1['DistrictCode_s'])
sf1['Council District']= sf1['Council District_f'].where(sf1['Council District_s'].isnull(), sf1['Council District_s'])
sf1['AddressCity']= sf1['AddressCity_f'].where(sf1['AddressCity_s'].isnull(), sf1['AddressCity_s'])
sf1['AddressPostalCode']= sf1['AddressPostalCode_f'].where(sf1['AddressPostalCode_s'].isnull(), sf1['AddressPostalCode_s'])
#sf1['FinishedArea']= sf1['FinishedArea_f'].where(sf1['FinishedArea_s'].isnull(), sf1['FinishedArea_s'])
sf1.columns

sf1.drop(['APN_f','DistrictCode_f','Council District_f','AddressCity_f',
          'AddressPostalCode_f','DistrictCode_s','Council District_s','AddressCity_s','APN_s','AddressPostalCode_s'], axis = 1,inplace = True)

final_sf = pd.merge(sf1, sf_2017,on= 'AddressFullAddress', how = 'outer', suffixes = ('_m','_n'),indicator = True)
final_sf.columns

final_sf.rename(columns = {'FinishedArea': '2017_FinishedArea'}, inplace = True)

final_sf['APN']= final_sf['APN_m'].where(final_sf['APN_n'].isnull(), final_sf['APN_n'])
final_sf['DistrictCode']= final_sf['DistrictCode_m'].where(final_sf['DistrictCode_n'].isnull(), final_sf['DistrictCode_n'])
final_sf['Council District']= final_sf['Council District_m'].where(final_sf['Council District_n'].isnull(), final_sf['Council District_n'])
final_sf['AddressCity']= final_sf['AddressCity_m'].where(final_sf['AddressCity_n'].isnull(), final_sf['AddressCity_n'])
final_sf['AddressPostalCode']= final_sf['AddressPostalCode_m'].where(final_sf['AddressPostalCode_n'].isnull(), final_sf['AddressPostalCode_n'])
#final_sf['FinishedArea']= final_sf['FinishedArea_m'].where(final_sf['FinishedArea_n'].isnull(), final_sf['FinishedArea_n'])


final_sf.drop(['APN_m','DistrictCode_m','Council District_m','AddressCity_m',
          'AddressPostalCode_m','DistrictCode_n','Council District_n','AddressCity_n','APN_n','AddressPostalCode_n'], axis = 1, inplace = True)
final_sf.columns

final_sf.drop_duplicates(keep ='first', inplace = True)

In [5]:
# Mortgage payment calculation
r = 0.04 #Annual interest rate
y = 30 #Length of mortgage (years)
n = 12*y # Number of monthly payments
d = 0.05 # Down payment percentage
final_sf['Mortgage_Per_Year_2009'] = 12* (1-d)*final_sf['2009 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2013'] = 12* (1-d)*final_sf['2013 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2017'] = 12* (1-d)*final_sf['2017 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)

In [6]:
import geopandas as gpd
council_district = gpd.read_file('data/Council_District_Outlines.geojson')

In [7]:
med_2013 = 62300
med_2017 = 68000
homes_2013 = 186747
homes_2017 = 197589
hh_2017 = 283929
pct_aff_low_2017 = 100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * .3 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]
pct_aff_high_2017 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * .6 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]) - pct_aff_low_2017
pct_wf_low_2017 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * .8 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017)
pct_wf_high_2017 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * 1.2 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017 + pct_wf_low_2017)
num_aff_low_2017 = homes_2017 * pct_aff_low_2017 / 100
num_aff_high_2017 = homes_2017 * pct_aff_high_2017 / 100
num_wf_low_2017 = homes_2017 * pct_wf_low_2017 / 100
num_wf_high_2017 = homes_2017 * pct_wf_high_2017 /100
pct_aff_low_2013 = 100 * final_sf.loc[final_sf.Mortgage_Per_Year_2013 < med_2013 * .3 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2013 > 0].shape[0]
pct_aff_high_2013 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2013 < med_2013 * .6 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2013 > 0].shape[0]) - pct_aff_low_2013
pct_wf_low_2013 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2013 < med_2013 * .8 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2013 > 0].shape[0]) - (pct_aff_low_2013 + pct_aff_high_2013)
pct_wf_high_2013 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2013 < med_2013 * 1.2 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2013 > 0].shape[0]) - (pct_aff_low_2013 + pct_aff_high_2013 + pct_wf_low_2013)
num_aff_low_2013 = homes_2013 * pct_aff_low_2013 / 100
num_aff_high_2013 = homes_2013 * pct_aff_high_2013 / 100
num_wf_low_2013 = homes_2013 * pct_wf_low_2013 / 100
num_wf_high_2013 = homes_2013 * pct_wf_high_2013 /100


def home_proj(hp_growth, income_growth, home_growth):
    med_2025 = med_2017* (1+income_growth)**8
    final_sf['Mortgage_Per_Year_2025'] = final_sf['Mortgage_Per_Year_2017']*(1 + hp_growth)**8
    
    pct_aff_low = 100 * final_sf.loc[final_sf.Mortgage_Per_Year_2025 < med_2025 * .3 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2025 > 0].shape[0]
    pct_aff_high = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2025 < med_2025 * .6 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2025 > 0].shape[0]) - pct_aff_low
    pct_wf_low = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2025 < med_2025 * .8 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2025 > 0].shape[0]) - (pct_aff_low + pct_aff_high)
    pct_wf_high = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2025 < med_2025 * 1.2 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2025 > 0].shape[0]) - (pct_aff_low + pct_aff_high + pct_wf_low)
        
    pct_aff = 100 * final_sf.loc[final_sf.Mortgage_Per_Year_2025 < med_2025 * .6 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2025 > 0].shape[0]
    pct_workforce = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2025 < med_2025 * 1.2 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2025 > 0].shape[0]) - pct_aff
    
    homes_2025 = homes_2017*(1 + home_growth)**8

    num_aff = homes_2025 * pct_aff / 100
    num_workforce = homes_2025 * pct_workforce / 100
    
    num_aff_low = homes_2025 * pct_aff_low / 100
    num_aff_high = homes_2025 * pct_aff_high / 100
    num_wf_low = homes_2025 * pct_wf_low / 100
    num_wf_high = homes_2025 * pct_wf_high /100
    
    #hh_2025 = hh_2017*(1+pop_growth)**8
    
    groups = ['Affordable 0% - 30%', 'Affordable 30% - 60%', 'Workforce 60% - 80%', 'Workforce 80% - 120%']
    pct_2013 = [pct_aff_low_2013, pct_aff_high_2013, pct_wf_low_2013, pct_wf_high_2013]
    num_2013 = [num_aff_low_2013, num_aff_high_2013, num_wf_low_2013, num_wf_high_2013]
    pct_2017 = [pct_aff_low_2017, pct_aff_high_2017, pct_wf_low_2017, pct_wf_high_2017]
    num_2017 = [num_aff_low_2017, num_aff_high_2017, num_wf_low_2017, num_wf_high_2017]
    pct_2025 = [pct_aff_low, pct_aff_high, pct_wf_low, pct_wf_high]
    num_2025 = [num_aff_low, num_aff_high, num_wf_low, num_wf_high]
    
    cum_pct_2013 = np.array(pct_2013).cumsum()
    cum_pct_2017 = np.array(pct_2017).cumsum()
    cum_pct_2025 = np.array(pct_2025).cumsum()
    
    df = pd.DataFrame({'Group' : groups, 'Percent 2013': pct_2013, 'Number 2013': num_2013,'Percent 2017': pct_2017, 'Number 2017' : num_2017,
                     'Percent 2025': pct_2025, 'Number 2025': num_2025})
    
    df['Number 2013'] = df['Number 2013'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Number 2017'] = df['Number 2017'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Number 2025'] = df['Number 2025'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Percent 2013'] = df['Percent 2013'].apply(lambda x: '{0:.2f}%'.format(x))
    df['Percent 2017'] = df['Percent 2017'].apply(lambda x: '{0:.2f}%'.format(x))
    df['Percent 2025'] = df['Percent 2025'].apply(lambda x: '{0:.2f}%'.format(x))

    
    #printmd('***Projected Number of Households in 2025: ***' + '{0:,.0f}'.format(hh_2025))
    printmd('***Projected Median Income in 2025 (Family of 4): ***' + '${:,.2f}'.format(med_2025))


    display(HTML(df.to_html(index = False)))
    
    #f, (a0, a1) = plt.subplots(1,2, gridspec_kw = {'width_ratios':[3, 1]})
    
    #fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    plt.figure(figsize=(12,8))    
    #fig = plt.figure(1)
    #fig.add_subplot(111)

    
    # y-axis in bold
    rc('font', weight='bold')

    # The position of the bars on the x-axis
    r = [0,1.5,3]

    # Names of group and bar width
    names = ['2013', '2017','2025']
    barWidth = 1

    labels = list(reversed(['0% - 30%', '30% - 60%', '60% - 80%', '80% - 120%']))
    
    plt.grid(axis = 'y', linestyle = '--', color = 'black')

    for i in range(len(cum_pct_2017)):
        plt.bar(r, [cum_pct_2013[len(cum_pct_2013) -1 - i],cum_pct_2017[len(cum_pct_2017) -1 - i], cum_pct_2025[len(cum_pct_2017) -1 - i]], edgecolor='white', width=barWidth, label = labels[i])

    # Custom X axis
    plt.xticks(r, names, fontweight='bold', fontsize = 16)
    plt.xlabel("Year", fontsize = 16)
    plt.ylabel('Percentage of Single Family Homes', fontsize = 16)
    plt.yticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], fontsize = 14)
    plt.title("Share of Homes by Price Range\n Davidson County", fontsize = 16)
    
    plt.legend(title = "Income Bracket \n (Percent of AMI)", loc = 'upper right', fontsize = 14)
        
  


In [8]:
style = {'description_width': 'initial'}
# Continuous_update = False for IntSlider control to stop continuous model evaluation while the slider is being dragged
m = interactive(home_proj,
                hp_growth=widgets.Dropdown(options={"High: 10 %" : 0.1,
                                                    "Recent: 9.2%":0.092,
                                                    "Medium: 7.5%":0.075,
                                                    "Average: 5.05%":0.0505},
                                          description="Growth in Home Prices",style=style, value = 0.092),
                income_growth =widgets.Dropdown(options={"High: 5.5%" : 0.055,
                                                         "Current: 4.5%":0.045,
                                                    "Medium: 3%":0.03,
                                                    "Average: 1.85%":0.0185},
                                          description="Income Growth",style=style, value = 0.045),
                pop_growth = widgets.Dropdown(options={"High: 2%":0.02,
                                                    "Medium: 1.5%":0.015,
                                                    "Low: 1%":0.01},
                                              value = 0.015,
                                          description="Population Growth",style=style),
                home_growth = widgets.Dropdown(options={"High: 1.8%":0.018,
                                                    "Recent: 1.4%":0.014,
                                                    "Low: 1%":0.01},
                                               value = 0.014,
                                          description="Growth in Home Supply",style=style)
)

# Set the height of the control.children[-1] so that the output does not jump and flicker
#output = m.children[-1]
#output.layout.height = '450px'

# Display the control
display(m)

interactive(children=(Dropdown(description='Growth in Home Prices', index=1, options={'High: 10 %': 0.1, 'Rece…

In [9]:
final_sf['Change_Per_Year'] = (final_sf.Mortgage_Per_Year_2017/final_sf.Mortgage_Per_Year_2013)**(1/4) -1

rates = dict(final_sf.groupby('Council District').Change_Per_Year.mean())

In [10]:
med_2013 = 62300
med_2017 = 68000
hh_2017 = 283929

def dist_proj(district, hp_growth, income_growth, home_growth):
    df = final_sf.loc[final_sf['Council District'] == district]
    med_2025 = med_2017* (1+income_growth)**8
     
    homes_2013 = df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]
    homes_2017 = df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]
       
    pct_aff_low_2013 = 100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * .3 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]
    pct_aff_high_2013 = (100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]) - pct_aff_low_2013
    pct_wf_low_2013 = (100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * .8 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]) - (pct_aff_low_2013 + pct_aff_high_2013)
    pct_wf_high_2013 = (100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]) - (pct_aff_low_2013 + pct_aff_high_2013 + pct_wf_low_2013)
    num_aff_low_2013 = homes_2013 * pct_aff_low_2013 / 100
    num_aff_high_2013 = homes_2013 * pct_aff_high_2013 / 100
    num_wf_low_2013 = homes_2013 * pct_wf_low_2013 / 100
    num_wf_high_2013 = homes_2013 * pct_wf_high_2013 /100
    
    pct_aff_low_2017 = 100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * .3 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]
    pct_aff_high_2017 = (100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]) - pct_aff_low_2017
    pct_wf_low_2017 = (100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * .8 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017)
    pct_wf_high_2017 = (100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017 + pct_wf_low_2017)
    num_aff_low_2017 = homes_2017 * pct_aff_low_2017 / 100
    num_aff_high_2017 = homes_2017 * pct_aff_high_2017 / 100
    num_wf_low_2017 = homes_2017 * pct_wf_low_2017 / 100
    num_wf_high_2017 = homes_2017 * pct_wf_high_2017 /100

    dist_num_homes = df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]
    dist_build_rate = (df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0])**(1/4) - 1
    
    
    if hp_growth == -9999:
        growth = rates[district]
    else:
        growth = hp_growth
    
    df['Mortgage_Per_Year_2025'] = final_sf['Mortgage_Per_Year_2017']*(1 + growth)**8
    
    pct_aff_low = 100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .3 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]
    pct_aff_high = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - pct_aff_low
    pct_wf_low = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .8 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - (pct_aff_low + pct_aff_high)
    pct_wf_high = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - (pct_aff_low + pct_aff_high + pct_wf_low)
        
    pct_aff = 100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]
    pct_workforce = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - pct_aff
    

    if home_growth == -9999:
        supply_growth = dist_build_rate
    else:
        supply_growth = home_growth
        
    homes_2025 = dist_num_homes*(1 + supply_growth)**8
    

    num_aff = homes_2025 * pct_aff / 100
    num_workforce = homes_2025 * pct_workforce / 100
    
    num_aff_low = homes_2025 * pct_aff_low / 100
    num_aff_high = homes_2025 * pct_aff_high / 100
    num_wf_low = homes_2025 * pct_wf_low / 100
    num_wf_high = homes_2025 * pct_wf_high /100
    
    #hh_2025 = hh_2017*(1+pop_growth)**8
    
    groups = ['Affordable 0% - 30%', 'Affordable 30% - 60%', 'Workforce 60% - 80%', 'Workforce 80% - 120%']
    pct_2013 = [pct_aff_low_2013, pct_aff_high_2013, pct_wf_low_2013, pct_wf_high_2013]
    num_2013 = [num_aff_low_2013, num_aff_high_2013, num_wf_low_2013, num_wf_high_2013]
    pct_2017 = [pct_aff_low_2017, pct_aff_high_2017, pct_wf_low_2017, pct_wf_high_2017]
    num_2017 = [num_aff_low_2017, num_aff_high_2017, num_wf_low_2017, num_wf_high_2017]
    pct_2025 = [pct_aff_low, pct_aff_high, pct_wf_low, pct_wf_high]
    num_2025 = [num_aff_low, num_aff_high, num_wf_low, num_wf_high]
    
    df = pd.DataFrame({'Group' : groups, 'Percent 2013': pct_2013, 'Number 2013': num_2013,'Percent 2017': pct_2017, 'Number 2017' : num_2017,
                     'Percent 2025': pct_2025, 'Number 2025': num_2025})
    
    df['Number 2013'] = df['Number 2013'].apply(lambda x: '{0:,.0f}'.format(x))    
    df['Number 2017'] = df['Number 2017'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Number 2025'] = df['Number 2025'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Percent 2013'] = df['Percent 2013'].apply(lambda x: '{0:.2f}%'.format(x))
    df['Percent 2017'] = df['Percent 2017'].apply(lambda x: '{0:.2f}%'.format(x))
    df['Percent 2025'] = df['Percent 2025'].apply(lambda x: '{0:.2f}%'.format(x))

    printmd('***Annual Price Growth for the District, 2013-2017: ***' + '{0:.2f}%'.format(100*rates[district]))
    printmd('***Single Family Home Supply in 2017: ***' + '{0:,.0f}'.format(dist_num_homes))
    printmd('***Supply Growth for the District, 2013 - 2017: ***' + '{0:.2f}%'.format(100*dist_build_rate))
    printmd('***Projected Single Family Home Supply in 2025: ***' + '{0:,.0f}'.format(homes_2025))
    printmd('***Median Income in 2025 (Family of 4): ***' + '${:,.2f}'.format(med_2025))


    display(HTML(df.to_html(index = False)))
    
    cum_pct_2013 = np.array(pct_2013).cumsum()
    cum_pct_2017 = np.array(pct_2017).cumsum()
    cum_pct_2025 = np.array(pct_2025).cumsum()
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))

    # y-axis in bold
    rc('font', weight='bold')

    # The position of the bars on the x-axis
    r = [0,1.5,3]

    # Names of group and bar width
    names = ['2013', '2017','2025']
    barWidth = 1

    labels = list(reversed(['0% - 30%', '30% - 60%', '60% - 80%', '80% - 120%']))
    
    plt.grid(axis = 'y', linestyle = '--', color = 'black')

    for i in range(len(cum_pct_2017)):
        plt.bar(r, [cum_pct_2013[len(cum_pct_2013) -1 - i],cum_pct_2017[len(cum_pct_2017) -1 - i], cum_pct_2025[len(cum_pct_2017) -1 - i]], edgecolor='white', width=barWidth, label = labels[i])

    # Custom X axis
    plt.xticks(r, names, fontweight='bold', fontsize = 16)
    plt.xlabel("Year", fontsize = 16)
    plt.yticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], fontsize = 14)
    plt.title(f"Share of Homes by Price Range\n District {district}", fontsize = 16)
    plt.ylabel('Percentage of Single Family Homes', fontsize = 16)
    plt.xlim(right=4.5)

    plt.legend(title = "Income Bracket \n (Percent of AMI)", loc = 'upper right', fontsize = 14)
    
    
    a = plt.axes([.75, .2, .3, .3])
    council_district['color'] = [1 if row[1].district == str(district) else 0 for row in council_district.iterrows()]
    council_district.plot(ax = a, column = 'color', cmap = 'binary', edgecolor = 'black')
    plt.axis('off')



In [11]:
style = {'description_width': 'initial'}
# Continuous_update = False for IntSlider control to stop continuous model evaluation while the slider is being dragged
m = interactive(dist_proj,
                district = widgets.Dropdown(options = list(range(1, 36)), description = 'District:', style = style),
                
                #dist_rate = rates[district.value],
                
                
                hp_growth=widgets.Dropdown(options={"High: 10 %" : 0.10,
                                                    "District Growth Rate" : -9999,
                                                    "Recent: 9.2%": 0.092,
                                                    "Medium: 7.5%":0.075,
                                                    "Average: 5.05%":0.0505},
                                          description="Growth in Home Prices",style=style, value = -9999),
                income_growth =widgets.Dropdown(options={"High: 5.5%" : 0.055,
                                                         "Current: 4.5%":0.045,
                                                    "Medium: 3%":0.03,
                                                    "Average: 1.85%":0.0185},
                                          description="Income Growth",style=style, value = 0.045),
                pop_growth = widgets.Dropdown(options={"High: 2%":0.02,
                                                    "Medium: 1.5%":0.015,
                                                    "Low: 1%":0.01},
                                              value = 0.015,
                                          description="Population Growth",style=style),
                home_growth = widgets.Dropdown(options={"District Growth Rate" : -9999,
                                                        "High: 1.8%":0.018,
                                                    "Recent: 1.4%":0.014,
                                                    "Low: 1%":0.01},
                                               value = -9999,
                                          description="Growth in Home Supply",style=style)
)

# Set the height of the control.children[-1] so that the output does not jump and flicker
#output = m.children[-1]
#output.layout.height = '450px'

# Display the control
display(m)

interactive(children=(Dropdown(description='District:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

In [12]:
city_rates = dict(final_sf.groupby('AddressCity').Change_Per_Year.mean())

In [13]:
med_2013 = 62300
med_2017 = 68000
#homes_2017 = 197589
hh_2017 = 283929
#pct_aff_low_2017 = 100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * .3 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]
#pct_aff_high_2017 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * .6 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]) - pct_aff_low_2017
#pct_wf_low_2017 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * .8 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017)
#pct_wf_high_2017 = (100 * final_sf.loc[final_sf.Mortgage_Per_Year_2017 < med_2017 * 1.2 * .3].shape[0] / final_sf.loc[final_sf.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017 + pct_wf_low_2017)
#num_aff_low_2017 = homes_2017 * pct_aff_low_2017 / 100
#num_aff_high_2017 = homes_2017 * pct_aff_high_2017 / 100
#num_wf_low_2017 = homes_2017 * pct_wf_low_2017 / 100
#num_wf_high_2017 = homes_2017 * pct_wf_high_2017 /100


def city_proj(city, hp_growth, income_growth, home_growth):
    df = final_sf.loc[final_sf['AddressCity'] == city.upper()]
    med_2025 = med_2017* (1+income_growth)**8
    
    dist_num_homes = df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]
    dist_build_rate = (df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0])**(1/4) - 1
    
    homes_2013 = df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]
    homes_2017 = df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]
    
    pct_aff_low_2013 = 100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * .3 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]
    pct_aff_high_2013 = (100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]) - pct_aff_low_2013
    pct_wf_low_2013 = (100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * .8 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]) - (pct_aff_low_2013 + pct_aff_high_2013)
    pct_wf_high_2013 = (100 * df.loc[df.Mortgage_Per_Year_2013 < med_2013 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2013 > 0].shape[0]) - (pct_aff_low_2013 + pct_aff_high_2013 + pct_wf_low_2013)
    num_aff_low_2013 = homes_2013 * pct_aff_low_2013 / 100
    num_aff_high_2013 = homes_2013 * pct_aff_high_2013 / 100
    num_wf_low_2013 = homes_2013 * pct_wf_low_2013 / 100
    num_wf_high_2013 = homes_2013 * pct_wf_high_2013 /100
    pct_aff_low_2017 = 100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * .3 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]
    pct_aff_high_2017 = (100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]) - pct_aff_low_2017
    pct_wf_low_2017 = (100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * .8 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017)
    pct_wf_high_2017 = (100 * df.loc[df.Mortgage_Per_Year_2017 < med_2017 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2017 > 0].shape[0]) - (pct_aff_low_2017 + pct_aff_high_2017 + pct_wf_low_2017)
    num_aff_low_2017 = homes_2017 * pct_aff_low_2017 / 100
    num_aff_high_2017 = homes_2017 * pct_aff_high_2017 / 100
    num_wf_low_2017 = homes_2017 * pct_wf_low_2017 / 100
    num_wf_high_2017 = homes_2017 * pct_wf_high_2017 /100

    
    if hp_growth == -9999:
        growth = city_rates[city.upper()]
    else:
        growth = hp_growth
        
    if home_growth == -9999:
        supply_growth = dist_build_rate
    else:
        supply_growth = home_growth
    
    df['Mortgage_Per_Year_2025'] = final_sf['Mortgage_Per_Year_2017']*(1 + growth)**8
    
    pct_aff_low = 100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .3 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]
    pct_aff_high = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - pct_aff_low
    pct_wf_low = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .8 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - (pct_aff_low + pct_aff_high)
    pct_wf_high = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - (pct_aff_low + pct_aff_high + pct_wf_low)
        
    pct_aff = 100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * .6 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]
    pct_workforce = (100 * df.loc[df.Mortgage_Per_Year_2025 < med_2025 * 1.2 * .3].shape[0] / df.loc[df.Mortgage_Per_Year_2025 > 0].shape[0]) - pct_aff
    
    homes_2025 = dist_num_homes*(1 + supply_growth)**8

    num_aff = homes_2025 * pct_aff / 100
    num_workforce = homes_2025 * pct_workforce / 100
    
    num_aff_low = homes_2025 * pct_aff_low / 100
    num_aff_high = homes_2025 * pct_aff_high / 100
    num_wf_low = homes_2025 * pct_wf_low / 100
    num_wf_high = homes_2025 * pct_wf_high /100
    
    #hh_2025 = hh_2017*(1+pop_growth)**8
    
    groups = ['Affordable 0% - 30%', 'Affordable 30% - 60%', 'Workforce 60% - 80%', 'Workforce 80% - 120%']
    pct_2013 = [pct_aff_low_2013, pct_aff_high_2013, pct_wf_low_2013, pct_wf_high_2013]
    num_2013 = [num_aff_low_2013, num_aff_high_2013, num_wf_low_2013, num_wf_high_2013]
    pct_2017 = [pct_aff_low_2017, pct_aff_high_2017, pct_wf_low_2017, pct_wf_high_2017]
    num_2017 = [num_aff_low_2017, num_aff_high_2017, num_wf_low_2017, num_wf_high_2017]
    pct_2025 = [pct_aff_low, pct_aff_high, pct_wf_low, pct_wf_high]
    num_2025 = [num_aff_low, num_aff_high, num_wf_low, num_wf_high]
    
    df = pd.DataFrame({'Group' : groups, 'Percent 2013': pct_2013, 'Number 2013': num_2013,'Percent 2017': pct_2017, 'Number 2017' : num_2017,
                     'Percent 2025': pct_2025, 'Number 2025': num_2025})
    
    df['Number 2013'] = df['Number 2013'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Number 2017'] = df['Number 2017'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Number 2025'] = df['Number 2025'].apply(lambda x: '{0:,.0f}'.format(x))
    df['Percent 2013'] = df['Percent 2013'].apply(lambda x: '{0:.2f}%'.format(x))
    df['Percent 2017'] = df['Percent 2017'].apply(lambda x: '{0:.2f}%'.format(x))
    df['Percent 2025'] = df['Percent 2025'].apply(lambda x: '{0:.2f}%'.format(x))

    printmd('***Annual Price Growth for the City, 2013-2017: ***' + '{0:.2f}%'.format(100*city_rates[city.upper()]))
    printmd('***Single Family Home Supply in 2017: ***' + '{0:,.0f}'.format(dist_num_homes))
    printmd('***Supply Growth for the City, 2013 - 2017: ***' + '{0:.2f}%'.format(100*dist_build_rate))
    printmd('***Projected Single Family Home Supply in 2025: ***' + '{0:,.0f}'.format(homes_2025))
    printmd('***Median Income in 2025 (Family of 4): ***' + '${:,.2f}'.format(med_2025))



    display(HTML(df.to_html(index = False)))
    
    cum_pct_2013 = np.array(pct_2013).cumsum()    
    cum_pct_2017 = np.array(pct_2017).cumsum()
    cum_pct_2025 = np.array(pct_2025).cumsum()
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))

    # y-axis in bold
    rc('font', weight='bold')

    # The position of the bars on the x-axis
    r = [0,1.5,3]

    # Names of group and bar width
    names = ['2013', '2017','2025']
    barWidth = 1

    labels = list(reversed(['0% - 30%', '30% - 60%', '60% - 80%', '80% - 120%']))
    
    plt.grid(axis = 'y', linestyle = '--', color = 'black')

    for i in range(len(cum_pct_2017)):
        plt.bar(r, [cum_pct_2013[len(cum_pct_2013) -1 - i], cum_pct_2017[len(cum_pct_2017) -1 - i], cum_pct_2025[len(cum_pct_2017) -1 - i]], edgecolor='white', width=barWidth, label = labels[i])

    # Custom X axis
    plt.xticks(r, names, fontweight='bold', fontsize = 16)
    plt.xlabel("Year", fontsize = 16)
    plt.yticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], fontsize = 14)
    plt.ylabel('Percentage of Single Family Homes', fontsize = 16)
    plt.title(f"Share of Homes by Price Range\n {city}", fontsize = 16)

    plt.legend(title = "Income Bracket \n (Percent of AMI)", loc = 'upper right', fontsize = 14)
    

    # Show graphic
    plt.show()    


In [14]:
cities = sorted([city.title() for city in list(final_sf.AddressCity.unique())])
style = {'description_width': 'initial'}
# Continuous_update = False for IntSlider control to stop continuous model evaluation while the slider is being dragged
m = interactive(city_proj,
                city = widgets.Dropdown(options = cities, description = 'City:', style = style, value = 'Nashville'),
                
                
                
                hp_growth=widgets.Dropdown(options={"High: 10 %" : 0.10,
                                                    "City Growth Rate" : -9999,
                                                    "Recent: 9.2%": 0.092,
                                                    "Medium: 7.5%":0.075,
                                                    "Average: 5.05%":0.0505},
                                          description="Growth in Home Prices",style=style, value = -9999),
                income_growth =widgets.Dropdown(options={"High: 5.5%" : 0.055,
                                                         "Current: 4.5%":0.045,
                                                    "Medium: 3%":0.03,
                                                    "Average: 1.85%":0.0185},
                                          description="Income Growth",style=style, value = 0.045),
                pop_growth = widgets.Dropdown(options={"High: 2%":0.02,
                                                    "Medium: 1.5%":0.015,
                                                    "Low: 1%":0.01},
                                              value = 0.015,
                                          description="Population Growth",style=style),
                home_growth = widgets.Dropdown(options={"City Growth Rate" : -9999,
                                                        "High: 1.8%":0.018,
                                                    "Recent: 1.4%":0.014,
                                                    "Low: 1%":0.01},
                                               value = -9999,
                                          description="Growth in Home Supply",style=style)
)

# Set the height of the control.children[-1] so that the output does not jump and flicker
#output = m.children[-1]
#output.layout.height = '450px'

# Display the control
display(m)

interactive(children=(Dropdown(description='City:', index=11, options=('Antioch', 'Ashland City', 'Bellevue', …